# **DSBA 6165 - Lab 1: EDA**

Professor: Khalil Khouy

TA: Aashi Sethiya

# Initial Steps

The first step is to import the libraries that you will use to analyze the data.

In [ ]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Before importing data into Jupyter notebook, you must import file into Collab.
**This step must be repeated whenever runtime restarts.**

When importing the data you have two options for how to name the data:

1. Use a non-descriptive name such as data. Only use if only 1 dataset is being imported
2. Use a descriptive name such as cars. This can be used in all scenarios.

In [ ]:
#Importing Dataset

dfRL = pd.read_csv("used_cars_data.csv")


The next step is to look at the data as a whole. Some of the methods include but are not limited to:
1. Looking at the top of the data
2. Looking at the bottom of the data
3. Looking at spread of data
4. Looking at information about the columns

In [ ]:
#Looking at top 5 Rows
dfRL.head()


In [ ]:
#Looking at bottom 5 rows
dfRL.tail()


Looking at the top and bottom of the data doesn't tell you much about the data as a whole. These steps can be useful when you don't know what the data looks like and looking at this can help give basic information about the data such as examples of what kind of data populates each of the columns.

The next two methods of looking at the data will give us a better idea of how the data looks as a whole.

In [ ]:
#Look at data spread including object variables (Statistical Overview)
dfRL.describe()

In [ ]:
#Explore cardinality
dfRL.nunique()

In [ ]:
#Look at data types, non-null counts and data types
dfRL.info()

The last step of just looking at the data is looking at nulls becuase Null values often can't be used by machine learning packages so its a good idea to see where the nulls are.

In [ ]:
#Null Counts per column
dfRL.isna().sum()

In [ ]:
#Null % per each column
dfRL.isna().sum() / len(dfRL) * 100


# Feature Engineering
Now that we have a general idea of what the data, the first step is to get rid of superflous columns.
1. S.No. is an ID column, these do nothing but tick for each row and thus arent useful.
2. New_Price is missing >85% of its data. This will make it hard to impute missing values.

In [ ]:
#Remove S.No. column from data
dfRL = dfRL.drop(['S.No.'], axis = 1)
#Remove New_Price column from data
dfRL = dfRL.drop(['New_Price'], axis = 1)

#look at updated data
dfRL.info()

The year column tells us how old the car is. It will be easier to preform calculations with this information if we create an age column to have the age of the car in its own column.

In [ ]:
from datetime import datetime
#Create todays year and then calculate age
current_year = datetime.now().year
dfRL['age'] = current_year - dfRL['Year']

#Look at top of data to see updated column
dfRL.head()

The name column is holding multiple pieces of information within it. It is telling us the Brand of the car and the model of the car. Let's pull those into additonal columns.

In [ ]:
dfRL['Brand'] = dfRL.Name.str.split().str.get(0)
dfRL['Model'] = dfRL.Name.str.split().str.get(1) + " " + dfRL.Name.str.split().str.get(2)
dfRL[['Name','Brand','Model']]

Lastly lets look a bit deeper into the columns we made to see if any brand names are incorrect

In [ ]:
#Names of all unique brands in Brand Columns
unique_names = dfRL['Brand'].unique()

#Number of unique brands
num_of_unq_brand = dfRL['Brand'].nunique()

print(unique_names, num_of_unq_brand)


We can see that there are two different unique names for the Isuzu brand and Land isn't a car brand (the correct name is Land Rover).

So lets fix that!

In [ ]:
dfRL["Brand"].replace({"ISUZU": "Isuzu","Land":"Land Rover"}, inplace=True)

The last step before we can preform EDA on the data is to seperate the numerical and categorical variables. But first there are a couple of columns that arent the correct type as the include units

In [ ]:
#Converting column types of Mileage , Engine , Power
dfRL['Mileage'] = dfRL.Mileage.str.split().str.get(0)
dfRL['Engine'] = dfRL.Engine.str.split().str.get(0)
dfRL['Power'] = dfRL.Power.str.split().str.get(0)

dfRL["Mileage"] = dfRL["Mileage"].astype(float)
dfRL["Engine"] = dfRL["Engine"].astype(float)
dfRL["Power"].replace({"null": None}, inplace=True)
dfRL["Power"] = dfRL["Power"].astype(float)

In [ ]:
#run describe again after correcting data types

dfRL.describe()

Let's also try binning on mileage to convert it into a categorical variable! We are going to do 4 bins to split the data into quarters

In [ ]:
#Bin Mileage

dfRL['Bin_Mileage'] = pd.cut(dfRL['Mileage'], bins = 4)

#Convert from categorical into str (This is so it can be used like our other categorical variables below)

dfRL["Bin_Mileage"] = dfRL["Bin_Mileage"].astype(str)

#outputs

print(dfRL['Bin_Milage'])

In [ ]:
#Seperate columns based on data type
cat_cols = dfRL.select_dtypes(include=['object','string']).columns
num_cols = dfRL.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

# EDA - Exploratory Data Analysis

One of the first steps of EDA is to create histograms and box plots for the numerical variables.


In [ ]:
for col in num_cols:
    print(col)
    print('Skew :', round(dfRL[col].skew(), 2))
    plt.figure(figsize = (15, 4))
    plt.subplot(1, 2, 1)
    dfRL[col].hist(grid=False)
    plt.ylabel('count')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=dfRL[col])
    plt.show()

Handle the Kilometers driven outlier

Next we will create count plots for the categorical variables!

In [ ]:
#remove kilometers driven outlier
Q1 = dfRL['Kilometers_Driven'].quantile(0.25)
Q3 = dfRL['Kilometers_Driven'].quantile(0.75)

IQR = Q3 - Q1 

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

dfRL = dfRL[(dfRL['Kilometers_Driven'] >= lower_bound) & (dfRL['Kilometers_Driven'] <= upper_bound)]



#rerun plots

for col in num_cols:
    print(col)
    print('Skew :', round(dfRL[col].skew(), 2))
    plt.figure(figsize = (15, 4))
    plt.subplot(1, 2, 1)
    dfRL[col].hist(grid=False)
    plt.ylabel('count')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=dfRL[col])
    plt.show()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize = (18, 18))
fig.suptitle('Bar plot for all categorical variables in the dataset')
sns.countplot(ax = axes[0, 0], x = 'Fuel_Type', data = dfRL, color = 'blue',
              order = dfRL['Fuel_Type'].value_counts().index);
sns.countplot(ax = axes[0, 1], x = 'Transmission', data = dfRL, color = 'blue',
              order = dfRL['Transmission'].value_counts().index);
sns.countplot(ax = axes[1, 0], x = 'Owner_Type', data = dfRL, color = 'blue',
              order = dfRL['Owner_Type'].value_counts().index);
sns.countplot(ax = axes[1, 1], x = 'Location', data = dfRL, color = 'blue',
              order = dfRL['Location'].value_counts().index);
sns.countplot(ax = axes[2, 0], x = 'Brand', data = dfRL, color = 'blue',
              order = dfRL['Brand'].value_counts().iloc[:20].index);
sns.countplot(ax = axes[2, 1], x = 'Model', data = dfRL, color = 'blue',
              order = dfRL['Model'].value_counts().iloc[:20].index);
axes[1][1].tick_params(labelrotation=45);
axes[2][0].tick_params(labelrotation=90);
axes[2][1].tick_params(labelrotation=90);

To do Bi-variate Analysis, the numerical variables with a large range need to be converted into logs so that they can be compared with other numerical variables

In [ ]:
# Function for log transformation of the column
def log_transform(data,col):
    for colname in col:
      if colname in dfRL.columns:
         data[colname + '_log'] = np.log(data[colname] + 1)
      else:
         print(f"Column '{colname}' not found in the dfRL.")
         return data


In [ ]:
#Log transformation of both columns (KM & $$)
log_transform(dfRL,['Kilometers_Driven','Price'])
#Plot of Log transformation of the feature 'Kilometers_Driven'
sns.distplot(dfRL["Kilometers_Driven_log"], axlabel="Kilometers_Driven_log");
#Drop old columns
dfRL = dfRL.drop(['Kilometers_Driven', 'Price'], axis = 1)

In [ ]:
#Plot of Log transformation of the feature 'Price'
sns.distplot(dfRL["Price_log"], axlabel="Price_log");

**Its time for Bi-variate Analysis!**

Let's dive deep into the realtionships between the categorical variables using stacked bar charts!

In [ ]:
plt.figure(figsize=(13,17))
#FILL IN
plt.show()


The plot above provides the following insights:

* The variable Year has a positive correlation with price and mileage
* A year has a Negative correlation with kilometers-Driven
* Mileage is negatively correlated with Power
* As power increases, mileage decreases
* Car with recent make is higher at prices. As the age of the car increases price decreases
* Engine and Power increase, and the price of the car increases


Normal bar charts are how we can look at the relationships between categorical and numerical variables



In [ ]:
fig, axarr = plt.subplots(4, 2, figsize=(12, 18))
dfRL.groupby('Location')['Price_log'].mean().sort_values(ascending=False).plot.bar(ax=axarr[0][0], fontsize=12)
axarr[0][0].set_title("Location Vs Price", fontsize=18)
dfRL.groupby('Transmission')['Price_log'].mean().sort_values(ascending=False).plot.bar(ax=axarr[0][1], fontsize=12)
axarr[0][1].set_title("Transmission Vs Price", fontsize=18)
dfRL.groupby('Fuel_Type')['Price_log'].mean().sort_values(ascending=False).plot.bar(ax=axarr[1][0], fontsize=12)
axarr[1][0].set_title("Fuel_Type Vs Price", fontsize=18)
dfRL.groupby('Owner_Type')['Price_log'].mean().sort_values(ascending=False).plot.bar(ax=axarr[1][1], fontsize=12)
axarr[1][1].set_title("Owner_Type Vs Price", fontsize=18)
dfRL.groupby('Brand')['Price_log'].mean().sort_values(ascending=False).head(10).plot.bar(ax=axarr[2][0], fontsize=12)
axarr[2][0].set_title("Brand Vs Price", fontsize=18)
dfRL.groupby('Model')['Price_log'].mean().sort_values(ascending=False).head(10).plot.bar(ax=axarr[2][1], fontsize=12)
axarr[2][1].set_title("Model Vs Price", fontsize=18)
dfRL.groupby('Seats')['Price_log'].mean().sort_values(ascending=False).plot.bar(ax=axarr[3][0], fontsize=12)
axarr[3][0].set_title("Seats Vs Price", fontsize=18)
dfRL.groupby('Car_Age')['Price_log'].mean().sort_values(ascending=False).plot.bar(ax=axarr[3][1], fontsize=12)
axarr[3][1].set_title("Car_Age Vs Price", fontsize=18)
plt.subplots_adjust(hspace=1.0)
plt.subplots_adjust(wspace=.5)
sns.despine()

Observations

* The price of cars is high in Coimbatore and less price in Kolkata and Jaipur
* Automatic cars have more price than manual cars.
* Diesel and Electric cars have almost the same price, which is maximum, and LPG cars have the lowest price
* First-owner cars are higher in price, followed by a second
* The third owner’s price is lesser than the Fourth and above
* Lamborghini brand is the highest in price
* Gallardocoupe Model is the highest in price
* 2 Seater has the highest price followed by 7 Seater
* The latest model cars are high in price


**Last but not least we have Multivariate Analysis**

A very common way to look at correlation between variables is through Heat maps.

In [ ]:
plt.figure(figsize=(12, 7))
# FILL IN
plt.show()

# Imputation of Variables

Since we have seats, engine, and power of cars that are missing we need a way to fill them in. Lets Go through Different ways of imputation!

In [ ]:
#Replace Seats with ....

#Replace Engine with ....

#Replace Power with ....


In [ ]:
#Test if it worked
